# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

>SQL:

```SQL
{{ config(materialized='table') }}

   SELECT extract(year from order_approved_at) AS ano_do_pedido, COUNT (order_id) AS total
     FROM `dbt_shop.orders` 
     WHERE order_approved_at IS NOT NULL
     GROUP BY ano_do_pedido
```

>Comando DBT 

` dbt run --models shop.order_by_year `

![Resposta](imagens_atividade3/exec1.png)

2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

>SQL:

```SQL
SELECT customers.customer_state AS estado, COUNT(orders.order_id) AS total
  FROM `dbt_shop.orders` AS orders
  INNER JOIN
    `dbt_shop.customers` AS customers ON (orders.customer_id = customers.customer_id)
  GROUP BY customers.customer_state
```

> Comando DBT

` dbt run --models shop.totais_pedidos_por_estado `

![Resposta](imagens_atividade3/exec2.png)

3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

> Query SQL:

```SQL
SELECT *
  FROM `dataset_rodrigo.totais_pedidos_por_estado`
  WHERE total > CAST('{{ var("number_of_orders") }}' AS INT64)
```

> Comando DBT

` dbt run --models shop.pedidos_por_estado_com_variavel --vars "{'number_of_orders': '5000'}" `

![Resposta](imagens_atividade3/exec3.png)

4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

> Query SQL:

```SQL
SELECT 
    CASE
        WHEN customer_state LIKE 'AM' THEN 'Norte'
        WHEN customer_state LIKE 'RR' THEN 'Norte'
        WHEN customer_state LIKE 'AP' THEN 'Norte'
        WHEN customer_state LIKE 'PA' THEN 'Norte'
        WHEN customer_state LIKE 'TO' THEN 'Norte'
        WHEN customer_state LIKE 'RO' THEN 'Norte'
        WHEN customer_state LIKE 'AC' THEN 'Norte'
        WHEN customer_state LIKE 'RN' THEN 'Nordeste'
        WHEN customer_state LIKE 'CE' THEN 'Nordeste'
        WHEN customer_state LIKE 'MA' THEN 'Nordeste'
        WHEN customer_state LIKE 'PI' THEN 'Nordeste'
        WHEN customer_state LIKE 'PE' THEN 'Nordeste'
        WHEN customer_state LIKE 'PB' THEN 'Nordeste'
        WHEN customer_state LIKE 'SE' THEN 'Nordeste'
        WHEN customer_state LIKE 'AL' THEN 'Nordeste'
        WHEN customer_state LIKE 'BA' THEN 'Nordeste'
        WHEN customer_state LIKE 'MS' THEN 'Centro-Oeste'
        WHEN customer_state LIKE 'GO' THEN 'Centro-Oeste'
        WHEN customer_state LIKE 'DF' THEN 'Centro-Oeste'
        WHEN customer_state LIKE 'SP' THEN 'Sudeste'
        WHEN customer_state LIKE 'RJ' THEN 'Sudeste'
        WHEN customer_state LIKE 'MG' THEN 'Sudeste'
        WHEN customer_state LIKE 'ES' THEN 'Sudeste'
        WHEN customer_state LIKE 'PR' THEN 'Sul'
        WHEN customer_state LIKE 'SC' THEN 'Sul'
        WHEN customer_state LIKE 'RS' THEN 'Sul'
    END AS regiao,
    COUNT (customer_id) AS numero_de_clientes
    FROM `dbt_shop.customers`
    GROUP BY regiao
```

> Comando DBT

` dbt run --models shop.customers_by_region `

![Resposta](imagens_atividade3/exec4.png)

5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

> Query SQL:

```SQL
{{ config(materialized='table') }}

   SELECT order_id, order_item_id, product_id, (price + freight_value) AS item_cost
     FROM `dbt_shop.items`
```

> Comando DBT

` dbt run --models shop.items_costs `

![Resposta](imagens_atividade3/exec5.png)

6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

> Query SQL:

```SQL
SELECT orders.order_id, orders.order_status, items.price, items.freight_value, (items.price + items.freight_value) AS full_price 
  FROM `dbt_shop.orders` AS orders
  INNER JOIN 
    `dbt_shop.items` AS items ON (items.order_id = orders.order_id)
  WHERE order_status = "shipped"
```

> Comando DBTl

` dbt run --models shop.orders_shipped_costs `

![Resposta](imagens_atividade3/exec6.png)